In [1]:
import torch
from transformers import AutoTokenizer, PhimoeConfig, PhimoeForCausalLM
from transformers.utils import logging
from accelerate import init_empty_weights, infer_auto_device_map

# Optional: Helps debugging
logging.set_verbosity_info()
device = torch.device("cuda:0")
# Model path
model_path = "microsoft/Phi-mini-MoE-instruct"

# Load config
config = PhimoeConfig.from_pretrained(model_path)
model = PhimoeForCausalLM.from_pretrained(model_path,config=config,
                                         device_map={"": device},
                                         torch_dtype=torch.float16,
                                         )
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()


🚨 `use_probabilistic_routing` is part of PhimoeModel.forward's signature, but not documented. Make sure to add it to the docstring of the function in /home/tkode/Desktop/transformers/src/transformers/models/phimoe/modeling_phimoe.py.
🚨 `prob_routing_temp` is part of PhimoeModel.forward's signature, but not documented. Make sure to add it to the docstring of the function in /home/tkode/Desktop/transformers/src/transformers/models/phimoe/modeling_phimoe.py.
🚨 `use_probabilistic_routing` is part of PhimoeForCausalLM.forward's signature, but not documented. Make sure to add it to the docstring of the function in /home/tkode/Desktop/transformers/src/transformers/models/phimoe/modeling_phimoe.py.
🚨 `prob_routing_temp` is part of PhimoeForCausalLM.forward's signature, but not documented. Make sure to add it to the docstring of the function in /home/tkode/Desktop/transformers/src/transformers/models/phimoe/modeling_phimoe.py.


loading configuration file config.json from cache at /home/tkode/.cache/huggingface/hub/models--microsoft--Phi-mini-MoE-instruct/snapshots/e8b3fcb41fef5b2e8644cffa99c83924db6a3ed4/config.json
Model config PhimoeConfig {
  "architectures": [
    "PhiMoEForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_slimmoe.PhiMoEConfig",
    "AutoModelForCausalLM": "modeling_slimmoe.PhiMoEForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "expert_dropout": 0.0,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_dropout": 0.0,
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "input_jitter_noise": 0.01,
  "intermediate_size": 960,
  "lm_head_bias": true,
  "max_position_embeddings": 4096,
  "model_type": "phimoe",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "num_local_experts": 16,
  "output_router_logits": false,
  "rms_norm_eps": 1e-05,
  "

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing PhimoeForCausalLM.

All the weights of PhimoeForCausalLM were initialized from the model checkpoint at microsoft/Phi-mini-MoE-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use PhimoeForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/tkode/.cache/huggingface/hub/models--microsoft--Phi-mini-MoE-instruct/snapshots/e8b3fcb41fef5b2e8644cffa99c83924db6a3ed4/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}

loading file tokenizer.model from cache at None
loading file tokenizer.json from cache at /home/tkode/.cache/huggingface/hub/models--microsoft--Phi-mini-MoE-instruct/snapshots/e8b3fcb41fef5b2e8644cffa99c83924db6a3ed4/tokenizer.json
loading file added_tokens.json from cache at /home/tkode/.cac

PhimoeForCausalLM(
  (model): PhimoeModel(
    (embed_tokens): Embedding(32064, 4096)
    (layers): ModuleList(
      (0-31): 32 x PhimoeDecoderLayer(
        (self_attn): PhimoeSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (block_sparse_moe): PhimoeSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=16, bias=False)
          (experts): ModuleList(
            (0-15): 16 x PhimoeBlockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=960, bias=False)
              (w2): Linear(in_features=960, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=960, bias=False)
              (act_fn): SiLU()
            )
          )
        )

In [ ]:
if "inputs" in globals():
    del inputs

prompt = 'I am an AI created in'
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# prob_routing_temp cannot be lower than about 0.0001. 
# 0.00001 causes cuda assert


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        use_probabilistic_routing=False,
        prob_routing_temp=0.001,
        #output_router_logits=True,
        #return_dict=True
    )

print(tokenizer.decode(outputs[0]))

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer